1\. Write a function that converts number representation (bin<->dec<->hex)

In [36]:
def base_converter(number, targetbase = 2):
    characters = list("0123456789abcdef") ## to be extended if you want to convert with sourcebase>16
    try:
        number[1]
    except:
        val10 = number
    else:
        if( number[1] == 'b' ): sourcebase = 2
        elif( number[1] == 'x' ): sourcebase = 16
        ## 'elif' to add other sourcebases, if needed.
        number = number[:1:-1]
        val10 = 0
        for i,j in enumerate(number):
            val10 += characters.index(j) * sourcebase**i
    
    if( targetbase == 10 ): return val10
    
    val = ""
    while( val10 > 0 ):
        rem = val10 % targetbase
        val = characters[rem] + val
        val10 = val10 // targetbase
    return val

test = bin(92)
print(test)
print(base_converter(test, 10))

0b1011100
92


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [76]:
def bin_to_f(string):
    if( len(string) != 32 ): raise Exception("String requested is not 32bits long")
    bias = 127
    sign = (-1)**int(string[0])
    mantissa = 1
    for i,j in enumerate(string[9:]):
        mantissa += int(j) * 2**(int(-1-i))
    exponent = int("0b"+string[1:9], 2)
    return sign * mantissa * 2**(exponent-bias)

test = "00000011111000000000000000000000"
bin_to_f(test)

1.316553672920962e-36

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [93]:
underflow, overflow = 0, 0
var = 1
while( var != 0 ):
    var = var / 2
    underflow += 1
print("The minimum float that can be represented is", 1/2**(underflow-1))
var = 1.
while( var < float("inf") ):
    var = var * 2
    overflow += 1.
print("The maximum float that can be represented is", 2**(overflow-1))

The minimum float that can be represented is 5e-324
The maximum float that can be represented is 8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [101]:
var = -5.420380183e15
underflow = 0
while( var != var + var / 2**underflow ):
    underflow += 1
print("The precision is", 1/2**underflow)

The precision is 5.551115123125783e-17


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [116]:
def quadratic_a(a, b, c):
    x1 = ( -b + (b**2 - 4*a*c)**0.5 ) / 2 / a
    x2 = ( -b - (b**2 - 4*a*c)**0.5 ) / 2 / a
    return x1,x2

def quadratic_b(a, b, c):
    x1 = 2 * c / ( -b - (b**2 - 4*a*c)**0.5)
    x2 = 2 * c / ( -b + (b**2 - 4*a*c)**0.5)
    return x1,x2

def quadratic_c(a, b, c):
    x1 = 2 * c / ( -b - (b**2 - 4*a*c)**0.5)
    x2 = ( -b - (b**2 - 4*a*c)**0.5 ) / 2 / a
    return x1,x2

### A)
print("Case a):", quadratic_a(0.001, 1000, 0.001))

### B)
print("Case b):", quadratic_b(0.001, 1000, 0.001))

### C)
print("Case c):", quadratic_c(0.001, 1000, 0.001))

print((15500031)**0.5)

Case a): (-9.999894245993346e-07, -999999.999999)
Case b): (-1.000000000001e-06, -1000010.5755125057)
Case c): (-1.000000000001e-06, -999999.999999)
3937.007874007874


The correct solutions should be $x_1=-10^{-6}$ and $x_2=-(10^6-10^{-6})$

In the case _a)_ we have a very accurate solution for $x_2$, but $x_1$ has a wrong value at the fifth significant digit. In the case _b)_ we have the opposite situation: a very precise value for $x_1$ and a wrong fifth significant digit for $x_2$

We can explain these results considering underflow problems. Since the term $4ac$ is very small (order $10^-6$) compared to $b^2$ (order $10^6$), the square root $\sqrt{b^2-4ac}\simeq b$. In particular we have no difference between the square root and $b$ up to the 12th digit, so when we subtract the values, we lost the majority of the mantissa, so we lose the majority of the precision in the computation. In fact, in the cases when we add up the square root and $b$, we still have information in the whole mantissa.

For this reason is a good idea to avoid subtracting the square root and the $b$ term, and in order to do this we can merge the formulas from case _a)_ and case _b)_ and have a precise solution with, at least, 12 valid digits.

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
